# Problem 1: Continuous Time Random Walk

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# Transition rate matrix from the assignment
Lambda = np.array([
    [0, 2/5, 1/5, 0, 0],      # o
    [0, 0, 3/4, 1/4, 0],      # a
    [1/2, 0, 0, 1/3, 0],      # b
    [0, 0, 1/3, 0, 2/3],      # c
    [0, 1/3, 0, 1/3, 0]       # d
])

nodes = ['o', 'a', 'b', 'c', 'd']
mapping = {name: i for i, name in enumerate(nodes)}
n = len(nodes)

# Calculate exit rates (w) and transition probability matrix (P)
w = np.sum(Lambda, axis=1)
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

print("Exit rates w:", w)
print("Transition matrix P:\n", P)

Exit rates w: [0.6        1.         0.83333333 1.         0.66666667]
Transition matrix P:
 [[0.         0.66666667 0.33333333 0.         0.        ]
 [0.         0.         0.75       0.25       0.        ]
 [0.6        0.         0.         0.4        0.        ]
 [0.         0.         0.33333333 0.         0.66666667]
 [0.         0.5        0.         0.5        0.        ]]


### a) & b) Return time for node 'a'

In [2]:
# --- Simulation ---
start_node = mapping['a']
n_sim = 10000
times = []

for i in range(n_sim):
    curr = start_node
    t = 0
    while True:
        # Time to next jump (exponential dist based on local rate)
        dt = -np.log(np.random.rand()) / w[curr]
        t += dt
        
        # Jump to next node based on P
        curr = np.random.choice(n, p=P[curr])
        
        if curr == start_node:
            times.append(t)
            break

print(f"Simulated avg return time (a): {np.mean(times):.4f}")

# --- Theoretical ---
# Use uniformization to find pi
w_star = np.max(w)
P_bar = Lambda / w_star
# Add self loops to make it stochastic
row_sums = np.sum(P_bar, axis=1)
P_bar += np.diag(1 - row_sums)

# Find stationary distribution pi
vals, vecs = np.linalg.eig(P_bar.T)
pi = vecs[:, np.isclose(vals, 1)].flatten().real
pi = pi / np.sum(pi)

# Theoretical return time = 1 / (pi_i * w_i)
theo_return = 1 / (pi[start_node] * w[start_node])
print(f"Theoretical return time (a): {theo_return:.4f}")

Simulated avg return time (a): 6.7853
Theoretical return time (a): 6.7083


### c) & d) Hitting time from 'o' to 'd'

In [ ]:

start = mapping['o']
target = mapping['d']
hit_times = []

for i in range(n_sim):
    curr = start
    t = 0
    if curr == target:
        hit_times.append(0)
        continue
        
    while curr != target:
        dt = -np.log(np.random.rand()) / w[curr]
        t += dt
        curr = np.random.choice(n, p=P[curr])
    
    hit_times.append(t)

print(f"Simulated avg hitting time (o->d): {np.mean(hit_times):.4f}")

# --- Theoretical ---
# System of equations: x = k + P_hat * x

remove_idx = [target]
keep_idx = [i for i in range(n) if i not in remove_idx]

P_hat = P[np.ix_(keep_idx, keep_idx)]
k = 1 / w[keep_idx] # Average holding times

# Solve (I - P_hat)x = k
x_vec = np.linalg.solve(np.eye(len(keep_idx)) - P_hat, k)

# Get value for 'o' (index 0 in original, which is index 0 in keep_idx)
o_idx_reduced = keep_idx.index(start)
print(f"Theoretical hitting time (o->d): {x_vec[o_idx_reduced]:.4f}")

Simulated avg hitting time (o->d): 10.6925
Theoretical hitting time (o->d): 10.7667


### e) Opinion Dynamics Simulation

In [4]:
# Build networkx graph to check structure
G = nx.DiGraph()
for i in range(n):
    for j in range(n):
        if P[i,j] > 0:
            G.add_edge(nodes[i], nodes[j])

# Check convergence conditions
cond = nx.condensation(G)
sinks = [n for n, d in cond.out_degree() if d == 0]
print(f"Sinks in condensation graph: {len(sinks)}")
print(f"Is graph aperiodic? {nx.is_aperiodic(G)}")

# Run dynamics
x = np.random.rand(n)
for t in range(50):
    x = P @ x

print("Final state:", x)
print("Consensus reached:", np.allclose(x, x[0]))

Sinks in condensation graph: 1
Is graph aperiodic? True
Final state: [0.65795664 0.65795661 0.65795672 0.65795652 0.65795675]
Consensus reached: True


### f) Variance of consensus

In [ ]:
# Variances: o=1, a=2, b=2, c=2, d=1
variances = np.array([1, 2, 2, 2, 1])
# 1. Calculate the Left Eigenvector of P 
vals, vecs = np.linalg.eig(P.T)
nu = vecs[:, np.isclose(vals, 1)].flatten().real
nu = nu / np.sum(nu)  # Normalize so sum(nu) = 1


true_variance = np.sum((nu**2) * variances)
print(f"Correct Theoretical Variance: {true_variance:.5f}")


sim_vals = []
for _ in range(5000):
    x = np.random.normal(0, np.sqrt(variances))
    
    for _ in range(100): 
        x = P @ x
    sim_vals.append(x[0]) 

print(f"Simulated Variance: {np.var(sim_vals):.5f}")

Correct Theoretical Variance: 0.36941
True Simulated Variance: 0.38091


### g) Remove edges (d,a), (d,c), (a,c), (b,c)

In [ ]:
Lambda_g = Lambda.copy()
# Map: o:0, a:1, b:2, c:3, d:4
Lambda_g[4, 1] = 0 # d->a
Lambda_g[4, 3] = 0 # d->c
Lambda_g[1, 3] = 0 # a->c
Lambda_g[2, 3] = 0 # b->c

# Recalculate P for this graph
w_g = np.sum(Lambda_g, axis=1)
P_g = np.zeros_like(Lambda_g)

for i in range(n):
    if w_g[i] > 0:
        P_g[i] = Lambda_g[i] / w_g[i]
    else:
        P_g[i, i] = 1 

print("New P matrix:\n", P_g)

# Check structure
G_g = nx.DiGraph()
for i in range(n):
    for j in range(n):
        if P_g[i,j] > 0:
            G_g.add_edge(nodes[i], nodes[j])

cond_g = nx.condensation(G_g)
sinks_g = [n for n, d in cond_g.out_degree() if d == 0]
print("Number of sink components:", len(sinks_g))

for s in sinks_g:
    print("Sink members:", cond_g.nodes[s]['members'])

# Run simulation
x = np.array([0.1, 0.2, 0.3, 0.4, 0.9])
for _ in range(100):
    x = P_g @ x
print("Final state:", x)

New P matrix:
 [[0.         0.66666667 0.33333333 0.         0.        ]
 [0.         0.         1.         0.         0.        ]
 [1.         0.         0.         0.         0.        ]
 [0.         0.         0.33333333 0.         0.66666667]
 [0.         0.         0.         0.         1.        ]]
Number of sink components: 2
Sink members: {'b', 'o', 'a'}
Sink members: {'d'}
Final state: [0.2        0.2        0.2        0.66666667 0.9       ]


### h) Remove edges (b,o) and (d,a)

In [8]:
Lambda_h = Lambda.copy()
Lambda_h[2, 0] = 0 # b->o
Lambda_h[4, 1] = 0 # d->a

w_h = np.sum(Lambda_h, axis=1)
P_h = np.zeros_like(Lambda_h)

for i in range(n):
    if w_h[i] > 0:
        P_h[i] = Lambda_h[i] / w_h[i]
    else:
        P_h[i, i] = 1

G_h = nx.DiGraph()
for i in range(n):
    for j in range(n):
        if P_h[i,j] > 0:
            G_h.add_edge(nodes[i], nodes[j])

cond_h = nx.condensation(G_h)
sinks_h = [n for n, d in cond_h.out_degree() if d == 0]
print("Number of sink components:", len(sinks_h))

# Simulation
x = np.random.rand(n)
for _ in range(100):
    x = P_h @ x
print("Final state:", x)

Number of sink components: 1
Final state: [0.4001635  0.21604617 0.76839815 0.03192885 0.76839815]
